In [1]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

In [2]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import os

**Read deta**

In [3]:
desktopData= pd.read_csv("Dyt-desktop.csv", index_col=0, na_values=['(NA)'])
tabletData = pd.read_csv("Dyt-tablet.csv", index_col=0, na_values=['(NA)'])

**A function to separate data columns**

In [4]:
def SeparateColumns(dataSetName):
    columns = defaultdict(list)
    with open(dataSetName, 'r') as f:
        reader = csv.reader(f, delimiter=';')
        headers = next(reader)
        column_nums = range(len(headers)) # Do NOT change to xrange
        for row in reader:
            for i in column_nums:
                
                columns[headers[i]].append(row[i])
    # Following line is only necessary if you want a key error for invalid column names
    return dict(columns)

In [5]:
def cleanData(data) :
    for col in data.columns.values:
        data[col] = data[col].astype('string')
    #----------
    for col in data.columns.values:
        data[col] = data[col].astype('float',errors = 'ignore')
    #-----------
    data['Gender']=data.Gender.map({'Male': 1, 'Female': 2})
    data['Dyslexia']=data.Dyslexia.map({'No': 0, 'Yes': 1})
    data['Nativelang']=data.Nativelang.map({'No': 0, 'Yes': 1})
    data['Otherlang']=data.Otherlang.map({'No': 0, 'Yes': 1})

desktop dataset

In [6]:
columns = SeparateColumns('Dyt-desktop.csv')
desktopData=pd.DataFrame.from_dict(columns)

desktopData

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,Male,No,Yes,7,10,10,0,10,1,0,...,0,0,0,17,2,0,2,0.117647,0,No
1,Female,Yes,Yes,13,12,12,0,12,1,0,...,4,0.114286,0,26,2,2,2,0.0769231,0.0769231,Yes
2,Female,No,Yes,7,6,6,0,6,1,0,...,4,0.114286,0,26,1,3,1,0.0384615,0.115385,No
3,Female,No,Yes,7,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,No
4,Female,No,Yes,8,4,4,0,4,1,0,...,1,25,0.05,26,2,2,2,0.0769231,0.0769231,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3639,Male,No,No,10,7,7,0,7,1,0,...,2,0.67,0.33,4,1,3,1,0.25,0.75,Yes
3640,Female,No,Yes,15,9,9,0,9,1,0,...,3,0.75,0.25,4,2,2,2,0.5,0.5,No
3641,Female,No,Yes,15,11,11,0,11,1,0,...,3,0.6,0.4,4,2,2,2,0.5,0.5,No
3642,Female,No,Yes,15,10,10,0,10,1,0,...,3,0.75,0.25,4,3,1,3,0.75,0.25,No


In [7]:
print(desktopData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Columns: 197 entries, Gender to Dyslexia
dtypes: object(197)
memory usage: 5.5+ MB
None


In [8]:
desktopData.describe()

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
count,3644,3644,3644,3644,3644,3644,3644,3644,3644,3644,...,3644,3644,3644,3644,3644,3644,3644,3644,3644,3644
unique,2,2,2,11,35,20,13,20,61,70,...,14,166,177,27,5,5,15,45,49,2
top,Male,No,Yes,10,5,1,0,0,1,0,...,4,0.114286,0,26,2,2,2,0,0.0769231,No
freq,1851,2674,2955,561,441,1048,2040,956,1666,2040,...,930,756,1436,1293,1353,1321,1330,622,554,3252


In [9]:
cleanData(desktopData)

desktopData.head()

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,0,1,7.0,10.0,10.0,0.0,10.0,1.0,0.0,...,0.0,0.000000,0.00,17.0,2.0,0.0,2.0,0.117647,0.000000,0
1,2,1,1,13.0,12.0,12.0,0.0,12.0,1.0,0.0,...,4.0,0.114286,0.00,26.0,2.0,2.0,2.0,0.076923,0.076923,1
2,2,0,1,7.0,6.0,6.0,0.0,6.0,1.0,0.0,...,4.0,0.114286,0.00,26.0,1.0,3.0,1.0,0.038462,0.115385,0
3,2,0,1,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00,1.0,0.0,0.0,0.0,0.000000,0.000000,0
4,2,0,1,8.0,4.0,4.0,0.0,4.0,1.0,0.0,...,1.0,25.000000,0.05,26.0,2.0,2.0,2.0,0.076923,0.076923,0


In [10]:
desktopData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Columns: 197 entries, Gender to Dyslexia
dtypes: float64(193), int64(4)
memory usage: 5.5 MB


tablet dataset

In [11]:
columns = SeparateColumns('Dyt-tablet.csv')
tabletData=pd.DataFrame.from_dict(columns)
tabletData.replace(["NULL"], np.nan, inplace = True)

tabletData

/tmp/ipykernel_79032/2067260201.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tabletData.replace(["NULL"], np.nan, inplace = True)


,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,Male,Yes,No,7,6,6,0,6,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,Female,Yes,No,7,7,7,0,7,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,Female,Yes,No,7,6,6,0,6,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,Male,Yes,No,7,5,5,0,5,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,Male,Yes,No,7,8,6,2,8,0.75,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,Male,Yes,No,17,13,13,0,13,1,0,...,35,0.11428571428571,0,26,4,0,26,0.15384615384615,0,No
1391,Female,Yes,Yes,17,9,9,0,9,1,0,...,35,0.11428571428571,0,26,4,0,26,0.15384615384615,0,No
1392,Male,Yes,Yes,17,10,10,0,10,1,0,...,35,0.11428571428571,0,27,3,2,27,0.11111111111111,0.074074074074074,No
1393,Female,Yes,Yes,17,11,11,0,11,1,0,...,35,0.11428571428571,0,26,4,0,26,0.15384615384615,0,No


In [12]:
tabletData['Nativelang'].unique()

array(['Yes', 'No'], dtype=object)

In [13]:
tabletData['Otherlang'].unique()

array(['No', 'Yes'], dtype=object)

In [14]:
cleanData(tabletData)

tabletData.head()

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,1,0,7.0,7.0,7.0,0.0,7.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,1,0,7.0,5.0,5.0,0.0,5.0,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,1,0,7.0,8.0,6.0,2.0,8.0,0.75,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [15]:
stateOfNUll= tabletData.isnull().any()
i = 0
for state in stateOfNUll :
    if(state):
        tabletData[stateOfNUll.index[i]].fillna(round(tabletData[stateOfNUll.index[i]].mean() , 4), inplace=True)
    i = i + 1

tabletData

/tmp/ipykernel_79032/509825635.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tabletData[stateOfNUll.index[i]].fillna(round(tabletData[stateOfNUll.index[i]].mean() , 4), inplace=True)


,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
1,2,1,0,7.0,7.0,7.0,0.0,7.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
2,2,1,0,7.0,6.0,6.0,0.0,6.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
3,1,1,0,7.0,5.0,5.0,0.0,5.0,1.00,0.00,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
4,1,1,0,7.0,8.0,6.0,2.0,8.0,0.75,0.25,...,46.8333,0.386300,0.5439,52.51,2.7851,7.9719,52.51,0.970900,2.225400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,1,1,0,17.0,13.0,13.0,0.0,13.0,1.00,0.00,...,35.0000,0.114286,0.0000,26.00,4.0000,0.0000,26.00,0.153846,0.000000,0
1391,2,1,1,17.0,9.0,9.0,0.0,9.0,1.00,0.00,...,35.0000,0.114286,0.0000,26.00,4.0000,0.0000,26.00,0.153846,0.000000,0
1392,1,1,1,17.0,10.0,10.0,0.0,10.0,1.00,0.00,...,35.0000,0.114286,0.0000,27.00,3.0000,2.0000,27.00,0.111111,0.074074,0
1393,2,1,1,17.0,11.0,11.0,0.0,11.0,1.00,0.00,...,35.0000,0.114286,0.0000,26.00,4.0000,0.0000,26.00,0.153846,0.000000,0


In [16]:
cols_with_missing = [col for col in tabletData.columns if tabletData[col].isnull().any()]

# Drop columns desktop data
reduced_desktopData = desktopData.drop(cols_with_missing, axis=1)

# Drop columns tablet data
reduced_tabletData = tabletData.drop(cols_with_missing, axis=1)

In [17]:
commonalityColumns = ['Gender','Nativelang','Otherlang','Age' , 'Dyslexia']
for i in  range(30):
    if((i>=0 and i<12) or (i>=13 and i<17) or i==21 or i==22 or i==29):
        commonalityColumns.append('Clicks'+str(i+1))
        commonalityColumns.append('Hits'+str(i+1))
        commonalityColumns.append('Misses'+str(i+1))
        commonalityColumns.append('Score'+str(i+1))
        commonalityColumns.append('Accuracy'+str(i+1))
        commonalityColumns.append('Missrate'+str(i+1))

reduced_desktopData=reduced_desktopData.loc[:,commonalityColumns]
reduced_tabletData=reduced_tabletData.loc[:,commonalityColumns]

In [18]:
y=reduced_desktopData['Dyslexia']
X=reduced_desktopData.loc[:, reduced_desktopData.columns != 'Dyslexia']

In [19]:
reduced_desktopData

,Gender,Nativelang,Otherlang,Age,Dyslexia,Clicks1,Hits1,Misses1,Score1,Accuracy1,...,Misses23,Score23,Accuracy23,Missrate23,Clicks30,Hits30,Misses30,Score30,Accuracy30,Missrate30
0,1,0,1,7.0,0,10.0,10.0,0.0,10.0,1.0,...,1.0,3.0,0.750000,0.250000,14.0,1.0,2.0,1.0,0.071429,0.142857
1,2,1,1,13.0,1,12.0,12.0,0.0,12.0,1.0,...,1.0,5.0,0.833333,0.166667,17.0,2.0,2.0,2.0,0.117647,0.117647
2,2,0,1,7.0,0,6.0,6.0,0.0,6.0,1.0,...,2.0,3.0,0.600000,0.400000,17.0,1.0,3.0,1.0,0.058824,0.176471
3,2,0,1,7.0,0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,0.750000,0.250000,8.0,0.0,1.0,0.0,0.000000,125.000000
4,2,0,1,8.0,0,4.0,4.0,0.0,4.0,1.0,...,3.0,2.0,0.400000,0.600000,17.0,0.0,4.0,0.0,0.000000,0.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3639,1,0,0,10.0,1,7.0,7.0,0.0,7.0,1.0,...,2.0,1.0,0.330000,0.670000,4.0,0.0,4.0,0.0,0.000000,1.000000
3640,2,0,1,15.0,0,9.0,9.0,0.0,9.0,1.0,...,1.0,7.0,0.880000,0.130000,4.0,2.0,2.0,2.0,0.500000,0.500000
3641,2,0,1,15.0,0,11.0,11.0,0.0,11.0,1.0,...,2.0,6.0,0.750000,0.250000,4.0,0.0,4.0,0.0,0.000000,1.000000
3642,2,0,1,15.0,0,10.0,10.0,0.0,10.0,1.0,...,1.0,9.0,0.900000,0.100000,4.0,3.0,1.0,3.0,0.750000,0.250000


In [20]:
reduced_desktopData.columns

Index(['Gender', 'Nativelang', 'Otherlang', 'Age', 'Dyslexia', 'Clicks1',
       'Hits1', 'Misses1', 'Score1', 'Accuracy1',
       ...
       'Misses23', 'Score23', 'Accuracy23', 'Missrate23', 'Clicks30', 'Hits30',
       'Misses30', 'Score30', 'Accuracy30', 'Missrate30'],
      dtype='object', length=119)

train and test with desktop dataset

In [21]:
X

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Misses23,Score23,Accuracy23,Missrate23,Clicks30,Hits30,Misses30,Score30,Accuracy30,Missrate30
0,1,0,1,7.0,10.0,10.0,0.0,10.0,1.0,0.0,...,1.0,3.0,0.750000,0.250000,14.0,1.0,2.0,1.0,0.071429,0.142857
1,2,1,1,13.0,12.0,12.0,0.0,12.0,1.0,0.0,...,1.0,5.0,0.833333,0.166667,17.0,2.0,2.0,2.0,0.117647,0.117647
2,2,0,1,7.0,6.0,6.0,0.0,6.0,1.0,0.0,...,2.0,3.0,0.600000,0.400000,17.0,1.0,3.0,1.0,0.058824,0.176471
3,2,0,1,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,0.750000,0.250000,8.0,0.0,1.0,0.0,0.000000,125.000000
4,2,0,1,8.0,4.0,4.0,0.0,4.0,1.0,0.0,...,3.0,2.0,0.400000,0.600000,17.0,0.0,4.0,0.0,0.000000,0.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3639,1,0,0,10.0,7.0,7.0,0.0,7.0,1.0,0.0,...,2.0,1.0,0.330000,0.670000,4.0,0.0,4.0,0.0,0.000000,1.000000
3640,2,0,1,15.0,9.0,9.0,0.0,9.0,1.0,0.0,...,1.0,7.0,0.880000,0.130000,4.0,2.0,2.0,2.0,0.500000,0.500000
3641,2,0,1,15.0,11.0,11.0,0.0,11.0,1.0,0.0,...,2.0,6.0,0.750000,0.250000,4.0,0.0,4.0,0.0,0.000000,1.000000
3642,2,0,1,15.0,10.0,10.0,0.0,10.0,1.0,0.0,...,1.0,9.0,0.900000,0.100000,4.0,3.0,1.0,3.0,0.750000,0.250000


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
#----RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train , y_train)
y_pred = rfc.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8911055694098088


train with desktop dataset

test with tablet dataset

In [24]:
yTest=reduced_tabletData['Dyslexia']
XTest=reduced_tabletData.loc[:, reduced_tabletData.columns != 'Dyslexia']

In [25]:
rfc2 = RandomForestClassifier()
rfc2.fit(X_train , y_train)
y_pred = rfc2.predict(XTest)
print("Accuracy:",metrics.accuracy_score(yTest, y_pred))

Accuracy: 0.8931899641577061


In [26]:
parameters_list={
    'criterion':['gini','entropy','log_loss'],
    'max_features':['sqrt','log2'],
    'oob_score':[True,False],
}

In [27]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(rfc,param_grid=parameters_list,cv=5,scoring='accuracy',verbose=3)

In [28]:
clf.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END criterion=gini, max_features=sqrt, oob_score=True;, score=0.894 total time=   0.7s
[CV 2/5] END criterion=gini, max_features=sqrt, oob_score=True;, score=0.895 total time=   0.7s
[CV 3/5] END criterion=gini, max_features=sqrt, oob_score=True;, score=0.895 total time=   0.6s
[CV 4/5] END criterion=gini, max_features=sqrt, oob_score=True;, score=0.898 total time=   0.6s
[CV 5/5] END criterion=gini, max_features=sqrt, oob_score=True;, score=0.893 total time=   0.7s
[CV 1/5] END criterion=gini, max_features=sqrt, oob_score=False;, score=0.894 total time=   0.6s
[CV 2/5] END criterion=gini, max_features=sqrt, oob_score=False;, score=0.895 total time=   0.6s
[CV 3/5] END criterion=gini, max_features=sqrt, oob_score=False;, score=0.895 total time=   0.4s
[CV 4/5] END criterion=gini, max_features=sqrt, oob_score=False;, score=0.898 total time=   0.3s
[CV 5/5] END criterion=gini, max_features=sqrt, oob_score=False;, score

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'oob_score': [True, False]},
             scoring='accuracy', verbose=3)

In [29]:
clf.best_params_

{'criterion': 'log_loss', 'max_features': 'sqrt', 'oob_score': True}

In [30]:
bModel=RandomForestClassifier(criterion='entropy',max_features='sqrt',oob_score=False)

In [31]:
bModel.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy')

In [32]:
y_pred=bModel.predict(XTest)

In [33]:
for i in XTest.columns:
  print(i)

Gender
Nativelang
Otherlang
Age
Clicks1
Hits1
Misses1
Score1
Accuracy1
Missrate1
Clicks2
Hits2
Misses2
Score2
Accuracy2
Missrate2
Clicks3
Hits3
Misses3
Score3
Accuracy3
Missrate3
Clicks4
Hits4
Misses4
Score4
Accuracy4
Missrate4
Clicks5
Hits5
Misses5
Score5
Accuracy5
Missrate5
Clicks6
Hits6
Misses6
Score6
Accuracy6
Missrate6
Clicks7
Hits7
Misses7
Score7
Accuracy7
Missrate7
Clicks8
Hits8
Misses8
Score8
Accuracy8
Missrate8
Clicks9
Hits9
Misses9
Score9
Accuracy9
Missrate9
Clicks10
Hits10
Misses10
Score10
Accuracy10
Missrate10
Clicks11
Hits11
Misses11
Score11
Accuracy11
Missrate11
Clicks12
Hits12
Misses12
Score12
Accuracy12
Missrate12
Clicks14
Hits14
Misses14
Score14
Accuracy14
Missrate14
Clicks15
Hits15
Misses15
Score15
Accuracy15
Missrate15
Clicks16
Hits16
Misses16
Score16
Accuracy16
Missrate16
Clicks17
Hits17
Misses17
Score17
Accuracy17
Missrate17
Clicks22
Hits22
Misses22
Score22
Accuracy22
Missrate22
Clicks23
Hits23
Misses23
Score23
Accuracy23
Missrate23
Clicks30
Hits30
Misses30
Score30

In [34]:
print("Accuracy:",metrics.accuracy_score(yTest, y_pred))

Accuracy: 0.8939068100358423


In [35]:
import pickle

In [36]:
with open('model.pkl', 'wb') as file:
    pickle.dump(bModel, file)